# Uso de otimizador SGD em classificador

In [1]:
import seaborn as sns

In [234]:
titanic_df = sns.load_dataset('titanic')

In [235]:
feature_names = ['pclass', 'female', 'age', 'fare'] # classe, feminino, idade, preço da passagem
titanic_df['female'] = titanic_df['sex'].map({'male': 0, 'female':1})
titanic_df['alive'] = titanic_df['alive'].map({'no': 0, 'yes':1})
titanic_df.dropna(subset=feature_names, inplace=True)

In [236]:
X = titanic_df[feature_names].to_numpy()
Y = titanic_df['alive'].to_numpy()

In [237]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

In [238]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
class ClassBin(nn.Module):
    # Construtor
    def __init__(self):
        super(ClassBin, self).__init__()
        self.linear1 = nn.Linear(4, 4)    # primeira hidden layer
        self.dropout1 = nn.Dropout(0.2)   # dropout layer
        self.linear2 = nn.Linear(4, 4)    # segunda hidden layer
        self.dropout2 = nn.Dropout(0.2)   # dropout layer
        self.linear3 = nn.Linear(4, 1)    # terceira hidden layer
        self.dropout3 = nn.Dropout(0.2)   # dropout layer
        self.sigmoid = nn.Sigmoid()

    # Propagação (Feed Forward)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout1(x)
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)
        x = F.relu(self.linear3(x))
        x = self.dropout3(x)
        x = self.sigmoid(x)
        return x

model = ClassBin()

In [240]:
loss_fn = nn.BCELoss()
batch_size = 32
max_epochs = 100
learning_rate = 0.001
momentum = 0.9

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [241]:
# numpy ==> torch.Tensor
X_train = torch.Tensor(x_train)
X_test = torch.Tensor(x_test)
Y_train = torch.Tensor(y_train)
Y_test = torch.Tensor(y_test)

# Data Set
train = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train, batch_size=batch_size ,shuffle=True)

test = TensorDataset(X_test, Y_test)
teste_loader = DataLoader(test, batch_size=batch_size)


In [242]:
model.train()

for epoch in range(max_epochs):

    for batch in train_loader:

        x = batch[0]
        y = batch[1]

        y_pred = model(x)

        loss = loss_fn(y_pred, y.unsqueeze(1))

        model.zero_grad()

        loss.backward()

        optimizer.step()
    
    print(f"epoca {epoch:2.0f}: loss = {loss.item():.3f}")

epoca  0: loss = 0.660
epoca  1: loss = 0.570
epoca  2: loss = 0.539
epoca  3: loss = 0.573
epoca  4: loss = 0.622
epoca  5: loss = 0.623
epoca  6: loss = 0.721
epoca  7: loss = 0.673
epoca  8: loss = 0.601
epoca  9: loss = 0.598
epoca 10: loss = 0.741
epoca 11: loss = 0.650
epoca 12: loss = 0.691
epoca 13: loss = 0.660
epoca 14: loss = 0.608
epoca 15: loss = 0.607
epoca 16: loss = 0.563
epoca 17: loss = 0.686
epoca 18: loss = 0.575
epoca 19: loss = 0.580
epoca 20: loss = 0.633
epoca 21: loss = 0.695
epoca 22: loss = 0.661
epoca 23: loss = 0.574
epoca 24: loss = 0.572
epoca 25: loss = 0.651
epoca 26: loss = 0.569
epoca 27: loss = 0.607
epoca 28: loss = 0.655
epoca 29: loss = 0.520
epoca 30: loss = 0.608
epoca 31: loss = 0.723
epoca 32: loss = 0.806
epoca 33: loss = 0.748
epoca 34: loss = 0.564
epoca 35: loss = 0.675
epoca 36: loss = 0.578
epoca 37: loss = 0.636
epoca 38: loss = 0.646
epoca 39: loss = 0.776
epoca 40: loss = 0.708
epoca 41: loss = 0.674
epoca 42: loss = 0.546
epoca 43: l

In [243]:
model.eval()
    
with torch.no_grad():  # Desabilitar gradientes durante avaliação
    train_pred = model(X_train)
    train_pred_binary = (train_pred > 0.5).float().flatten()
    train_acc = torch.sum(train_pred_binary == Y_train) / len(Y_train)

    test_pred = model(X_test)
    test_pred_binary = (test_pred > 0.5).float().flatten()
    test_acc = torch.sum(test_pred_binary == Y_test) / len(Y_test)

print(f"Acurácia de Treino: {train_acc:.6f}")
print(f"Acurácia de Teste: {test_acc:.6f}")

Acurácia de Treino: 0.695271
Acurácia de Teste: 0.692308
